In [14]:
# Import libraries
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
from datetime import datetime, date, timedelta
from pytz import timezone
import sqlalchemy

# The function to get the Icao codes from the last Notebook
city = ['Berlin', 'Paris', 'Madrid']
latitude = [52.52000, 48.85667, 40.41694]
longitude = [13.40500, 2.35222, -3.70333]
data_df = pd.DataFrame()
data_df['city'] = city
data_df['latitude'] = latitude
data_df['longitude'] = longitude
def get_airports_loop(data_df):
    icao_list = []
  
    url = "https://aviation-reference-data.p.rapidapi.com/airports/search"
    headers = {
        "X-RapidAPI-Key": "API Key",
        "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
    }
    for i in range(len(data_df)):
        querystring = {"lat": data_df['latitude'][i], "lon": data_df['longitude'][i], "radius": "100"}
        response = requests.get(url, headers=headers, params=querystring)
        airports_json = response.json()

        for j in airports_json:
            icao_list.append(j.get('icaoCode', ''))
    return (icao_list)
icao_list = get_airports_loop(data_df)

# Function to get the flights from tomorrow
def tomorrows_flight_arrivals(icao_list):
  # Get today's date in Berlin timezone
  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  # Calculate tomorrow's date
  tomorrow = (today + timedelta(days=1))

  # Initialize an empty list to store flight data
  list_for_df = []

  # Loop over each ICAO code in the input list
  for icao in icao_list:
    if icao is None: continue
    # Define the two time periods for which to fetch data
    times = [["00:00","11:59"],["12:00","23:59"]]

    # Loop over each time period
    for time in times:
      # Construct the URL for the API request

      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      # Define the query parameters for the API request
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      # Define the headers for the API request
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': '0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276'
          }
      # Make the API request
      response = requests.request("GET", url, headers=headers, params=querystring)
      if response.status_code == 204 : continue
      # Parse the JSON response
      flights_json = response.json()

      # Loop over each flight in the response
      for flight in flights_json['arrivals']:
        # Initialize an empty dictionary to store flight data
        flights_dict = {}
        # Store the ICAO code and flight data in the dictionary
        flights_dict['arrival_icao'] = icao
        # Use the .get() method to avoid KeyError if a key doesn't exist in the dictionary
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        # Store the current date in Berlin timezone
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        # Append the flight dictionary to the list
        list_for_df.append(flights_dict)

  # Convert the list of flight dictionaries to a DataFrame and return it
  return pd.DataFrame(list_for_df)

cities_airports_flights_tomorrow = tomorrows_flight_arrivals(icao_list)

# Clean data
cities_airports_flights_tomorrow['arrival_time_local'] = pd.to_datetime(cities_airports_flights_tomorrow['arrival_time_local'])
cities_airports_flights_tomorrow['data_retrieved_on'] = pd.to_datetime(cities_airports_flights_tomorrow['data_retrieved_on'])

# Copying the dataframe to MySQL
schema="mydb" 
host="127.0.0.1"
user="root"
password="*******"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

cities_airports_flights_tomorrow.to_sql('flights', 
                                if_exists='append',
                                con=con,
                                index=False)
